In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Aesthetic Medicine Assistant: A Generative AI Solution for Healthcare

 ### Project Overview
 The Aesthetic Medicine Assistant is a comprehensive generative AI solution designed to
 assist healthcare professionals in the field of aesthetic medicine. This project leverages
 multiple generative AI capabilities to enhance patient consultations, treatment planning,
 and outcomes visualization in aesthetic procedures such as dermal fillers, botox, facial
 rejuvenation, and cosmetic dermatology.

### 🔧 [1] Setup & API Configuration

In [2]:
# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qy jupyterlab jupyterlab-lsp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.5 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types
from kaggle_secrets import UserSecretsClient
from IPython.display import Markdown, display

genai.__version__

'1.7.0'

In [4]:
from kaggle_secrets import UserSecretsClient

# GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


### 🧠 [2] Assistant Prompt Function

In [5]:
system_instruction = """
You are AesMed, a helpful and ethical assistant specialized in aesthetic medicine. 
You provide post-treatment care guidance, explain procedures, and highlight potential safety concerns.
You do not diagnose or prescribe. Recommend consulting certified professionals for serious issues.
"""

def ask_assistant(user_input: str, model="gemini-2.0-flash"):
    config = types.GenerateContentConfig(
        temperature=0.7,
        top_p=1.0,
        system_instruction=system_instruction,
        max_output_tokens=500
    )

    response = client.models.generate_content(
        model=model,
        config=config,
        contents=user_input
    )

    return response.text


### 📚 [3] Knowledge Base + Simulated RAG

In [6]:
knowledge_base = [
    "Retinol can increase skin sensitivity. Avoid using it immediately after a chemical peel.",
    "After a chemical peel, apply soothing products like hyaluronic acid or aloe vera.",
    "Sunscreen is critical after peels or laser treatments. Use SPF 50+ daily.",
    "Microneedling can improve fine lines and acne scars over several sessions.",
    "Botox temporarily relaxes muscles and typically lasts 3–4 months.",
]


In [7]:
def retrieve_relevant_docs(query, kb, top_k=2):
    query_words = query.lower().split()
    scored_docs = []

    for doc in kb:
        score = sum(word in doc.lower() for word in query_words)
        scored_docs.append((score, doc))

    scored_docs.sort(reverse=True)
    return [doc for score, doc in scored_docs[:top_k] if score > 0]

def rag_prompt(user_query):
    context = retrieve_relevant_docs(user_query, knowledge_base)
    if not context:
        return f"No related documents found. Please consult a licensed professional.\n\nQuestion: {user_query}"

    return f"""Use the following medical context to answer the user's question accurately.

Context:
{chr(10).join(context)}

Question:
{user_query}
"""
def rag_respond(user_query):
    prompt = rag_prompt(user_query)
    return ask_assistant(prompt)


### 🧭 [4] Function Calling + Intent Routing

In [8]:
function_calling_prompt = """
You are a routing assistant for an aesthetic medicine bot.

Functions:
- explain_procedure(name)
- post_care_recommendation()
- safety_warning(product)

Respond with the function call in Python-style syntax.

Examples:
Input: I had a microneedling session. What does it do?
Output: explain_procedure("microneedling")

Input: Should I use retinol after a chemical peel?
Output: safety_warning("retinol")

Input: What should I do after laser treatment?
Output: post_care_recommendation()
"""


In [9]:
def explain_procedure(name):
    return f"{name.title()} is a common aesthetic treatment. It may involve preparation, downtime, and aftercare. Always consult a licensed practitioner."

def post_care_recommendation():
    return "Apply soothing agents like aloe vera after treatment. Avoid sun and use SPF 50+ sunscreen."

def safety_warning(product):
    return f"Warning: {product.title()} may cause irritation if used after intensive skin treatments. Always follow professional guidance."


In [10]:
def route_query(query):
    config = types.GenerateContentConfig(
        temperature=0.0,
        system_instruction=function_calling_prompt
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        config=config,
        contents=query
    )
    return response.text.strip()

def execute_routed_function(code_string):
    try:
        if "explain_procedure" in code_string:
            arg = code_string.split('"')[1]
            return explain_procedure(arg)
        elif "safety_warning" in code_string:
            arg = code_string.split('"')[1]
            return safety_warning(arg)
        elif "post_care_recommendation" in code_string:
            return post_care_recommendation()
        else:
            return "Sorry, I couldn't determine what to do with that request."
    except:
        return "Function execution error."


### 📊 [5] Evaluator Block (Structured Output)

In [11]:
evaluation_prompt_template = """
You are an evaluator. Rate the quality of the AI's answer using these:

Criteria:
- Instruction following
- Groundedness
- Fluency

Ratings:
5 - Excellent
4 - Good
3 - Average
2 - Poor
1 - Unacceptable

Input:
Question: {question}
Answer: {answer}

Respond in this format:
Score: <1-5>
Explanation: <reason>
"""


In [12]:
def evaluate_response(question, answer):
    prompt = evaluation_prompt_template.format(question=question, answer=answer)

    config = types.GenerateContentConfig(temperature=0.0)

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        config=config,
        contents=prompt
    )
    return response.text.strip()


## 🧠 Main Chatbot Loop
This loop will:


* Allow real-time interaction via input() (or widgets if needed)
* Offer both RAG responses and Function-Routing responses
* Optionally run evaluation scoring on each response


In [ ]:
print("🤖 Welcome to AesMed — your Aesthetic Medicine Assistant.")
print("Ask a question, or type 'exit' to quit.\n")

while True:
    user_query = input("💬 You: ")

    if user_query.lower() in {"exit", "quit"}:
        print("🫡 AesMed: Thank you for using AesMed. Stay radiant!")
        break

    print("\n--- [RAG Response] ---")
    rag_reply = rag_respond(user_query)
    print(f"📚 AesMed (Contextual):\n{rag_reply}\n")

    print("--- [Function Calling] ---")
    function_code = route_query(user_query)
    print(f"🧠 Routed Function: {function_code}")
    print(f"⚙️  Assistant Output:\n{execute_routed_function(function_code)}\n")

    print("--- [Evaluation] ---")
    evaluation = evaluate_response(user_query, rag_reply)
    print(f"📊 Score:\n{evaluation}\n")
    print("=" * 80)


🤖 Welcome to AesMed — your Aesthetic Medicine Assistant.
Ask a question, or type 'exit' to quit.



💬 You:  Can I use retinol after a peel?



--- [RAG Response] ---
📚 AesMed (Contextual):
It's best to avoid using retinol immediately after a chemical peel, as peels can increase skin sensitivity.


--- [Function Calling] ---
🧠 Routed Function: safety_warning("retinol")
⚙️  Assistant Output:

--- [Evaluation] ---
📊 Score:
Score: 5
Explanation: The answer is concise, clear, and directly addresses the question with a helpful recommendation. It also provides a valid reason for the recommendation.



💬 You:  Should I use retinol after a peel?



--- [RAG Response] ---
📚 AesMed (Contextual):
It's best to avoid using retinol immediately after a chemical peel. Retinol can increase skin sensitivity, and peels already make your skin more sensitive than usual.


--- [Function Calling] ---
🧠 Routed Function: safety_warning("retinol")
⚙️  Assistant Output:

--- [Evaluation] ---
📊 Score:
Score: 5
Explanation: The answer is clear, concise, and directly addresses the question. It provides a sound recommendation based on the known effects of retinol and chemical peels on skin sensitivity.

